UNEMPLOYMENT -  LU_PE_NUM
EMPLOYMENT - LE_PE_NUM
POPULATION - LP_PE_NUM
GDP - NC_GDP_PT

Also, please be aware of the following rate limits and throttle your requests accordingly:

    10 requests in 5 second window from one user (IP)
    50 requests per second overall on the application

In [6]:
pip install psycopg2-binary

     |████████████████████████████████| 1.6MB 805kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests
import pprint
import pandas as pd
import config as cfg
import time
import sys
from sqlalchemy import create_engine
from datetime import datetime
from pprint import pprint

In [ ]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'

In [39]:
def getJsonCountry(index, country):
    key = f'CompactData/IFS/A.{country}.{index}'
    data = requests.get(f'{url}{key}').json()
    return data['CompactData']['DataSet']['Series']

def getDataSets():
    key = 'Dataflow'
    data = request.get('f{}')


def createDataList(json):
    data = json['Obs']
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    if isinstance(data, list):
        data_list = [(json.get('@REF_AREA'), json.get('@UNIT_MULT'), obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE'), timestamp) for obs in data]
    else:
        data_list = [(json.get('@REF_AREA'), json.get('@UNIT_MULT'), data.get('@TIME_PERIOD'), data.get('@OBS_VALUE'))]
    
    return data_list

def getDataStructure(dataSet):
    key = f"DataStructure/{dataSet}"
    dimension_list = requests.get(f'{url}{key}').json()
    return dimension_list

def getCountrylistIFS():
    codes = getDataStructure('IFS')["Structure"]["CodeLists"]["CodeList"]
    countries = [(country["@value"], country["Description"]["#text"]) for country in codes[2]['Code']]
    return countries
    

def insertIntoTable(table, tblList):
    tblList = ','.join([str(obs) for obs in tblList])  
    engine.execute("INSERT INTO " + table +" VALUES " + tblList)

def getSqlTableCreateStr(tblName, **fields):
    sqlstr = ', '.join([' '.join([key, str(val)]) for key, val in fields.items()])
    sqlstr = f'CREATE TABLE IF NOT EXISTS {tblName}({sqlstr})' 
    return sqlstr

In [32]:
def insertIntoCountries(countryList, IndexName, secsWait, engine):

    engine.execute(getSqlTableCreateStr(IndexName, REF_AREA='VARCHAR(5)', UNIT_MULT='INT', TIME_PERIOD='INT', VALUE='NUMERIC(18,5)', TIMESTAMP='TIMESTAMP'))

    for country in countryList:
    
        try:
            datacountry = getJsonCountry(IndexName,country)
            datacountry = createDataList(datacountry)
            insertIntoTable(IndexName, datacountry)
            print(f"Country {country} exctracted sucessfully for Indicator {IndexName}")
        except:
            print(f"Error ocurred while processing {IndexName} for {country} : {sys.exc_info()[0]}")

        time.sleep(secsWait)


In [40]:
pprint(getDataSets())

JSONDecodeError: Unexpected UTF-8 BOM (decode using utf-8-sig): line 1 column 1 (char 0)

---------------------------------------------------
INPUT DATA
- LIST COUNTRIES
- LIST INDEXES
----------------------------------------------------

In [33]:
#countries = getCountrylistIFS()
#abr_countries = [country[0] for country in countries]
countries = ['MX','DE','RU','BR','CO']
indexes = ['LU_PE_NUM', 'LE_PE_NUM', 'LP_PE_NUM', 'NGDP_XDC']

In [34]:
con_string = 'postgresql://postgres:' + cfg.dbpassword + '@localhost:5432/' + cfg.dbname
engine = create_engine(con_string)
connection = engine.connect()

In [35]:

for index in indexes:
    insertIntoCountries(countries, index, 3, engine)


Country MX exctracted sucessfully for Indicator LU_PE_NUM
Country DE exctracted sucessfully for Indicator LU_PE_NUM
Country RU exctracted sucessfully for Indicator LU_PE_NUM
Country BR exctracted sucessfully for Indicator LU_PE_NUM
Country CO exctracted sucessfully for Indicator LU_PE_NUM
Country MX exctracted sucessfully for Indicator LE_PE_NUM
Country DE exctracted sucessfully for Indicator LE_PE_NUM
Country RU exctracted sucessfully for Indicator LE_PE_NUM
Country BR exctracted sucessfully for Indicator LE_PE_NUM
Country CO exctracted sucessfully for Indicator LE_PE_NUM
Country MX exctracted sucessfully for Indicator LP_PE_NUM
Country DE exctracted sucessfully for Indicator LP_PE_NUM
Country RU exctracted sucessfully for Indicator LP_PE_NUM
Country BR exctracted sucessfully for Indicator LP_PE_NUM
Country CO exctracted sucessfully for Indicator LP_PE_NUM
Country MX exctracted sucessfully for Indicator NGDP_XDC
Country DE exctracted sucessfully for Indicator NGDP_XDC
Country RU exctr

In [ ]:
dataStructure = getDataStructure("IFS")